In [1]:
#Use 3.12.4
import os
import markdown
import torch
from bs4 import BeautifulSoup
from huggingface_hub import login
from sentence_transformers import SentenceTransformer

from langchain import FAISS
from langchain.text_splitter import SpacyTextSplitter 
from langchain.document_loaders import PyMuPDFLoader, DirectoryLoader, TextLoader
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.chat_models.base import BaseChatModel
from langchain.schema import HumanMessage, AIMessage, SystemMessage

from langchain.docstore.in_memory import InMemoryDocstore
from langchain.docstore.document import Document
import faiss
import numpy as np

key_path = '/Users/jaesolshin/key/HF_TOKEN.txt'
os.environ["HF_TOKEN"] = open(key_path, 'r', encoding='utf-8').read()
login(os.environ["HF_TOKEN"], add_to_git_credential=True)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /Users/jaesolshin/.cache/huggingface/token
Login successful


In [2]:
#!pip install pymupdf
from langchain.document_loaders import PyMuPDFLoader

path = '/Users/jaesolshin/Library/CloudStorage/GoogleDrive-ysys143@gmail.com/내 드라이브/2024-2/Google ML Bootcamp2024/data/bokbot/annual_report/2023년도 연차보고서.pdf'
loader = PyMuPDFLoader(path)
documents = loader.load()

In [3]:
print(f'문서개수: {len(documents)}')
print(f'첫번째 문서의 내용: {documents[0].page_content}')
print(f'첫번째 문서의 메타데이터: {documents[0].metadata}')

문서개수: 191
첫번째 문서의 내용: ISSN 1975-4922
2023
연차보고서
2024. 3

첫번째 문서의 메타데이터: {'source': '/Users/jaesolshin/Library/CloudStorage/GoogleDrive-ysys143@gmail.com/내 드라이브/2024-2/Google ML Bootcamp2024/data/bokbot/annual_report/2023년도 연차보고서.pdf', 'file_path': '/Users/jaesolshin/Library/CloudStorage/GoogleDrive-ysys143@gmail.com/내 드라이브/2024-2/Google ML Bootcamp2024/data/bokbot/annual_report/2023년도 연차보고서.pdf', 'page': 0, 'total_pages': 191, 'format': 'PDF 1.6', 'title': '', 'author': '1120077', 'subject': '', 'keywords': '', 'creator': 'Hwp 2018 10.0.0.10640', 'producer': 'Hancom PDF 1.3.0.538', 'creationDate': "D:20240328151525+09'00'", 'modDate': "D:20240619161741+09'00'", 'trapped': ''}


In [4]:
from langchain.text_splitter import SpacyTextSplitter #자연어 처리를 위한 여러 기능을 제공한다.
text_splitter = SpacyTextSplitter(
    chunk_size=300, #분할할 크기를 설정
    chunk_overlap=50,
    pipeline='ko_core_news_sm' #분할에 사용할 언어모델을 설정
)

splitted_documents = text_splitter.split_documents(documents) #문서를 분할

print(f'분할 전 문서 개수: {len(documents)}')
print(f'분할 후 문서 개수: {len(splitted_documents)}')

Created a chunk of size 318, which is longer than the specified 300
Created a chunk of size 434, which is longer than the specified 300
Created a chunk of size 881, which is longer than the specified 300
Created a chunk of size 342, which is longer than the specified 300
Created a chunk of size 412, which is longer than the specified 300
Created a chunk of size 323, which is longer than the specified 300
Created a chunk of size 367, which is longer than the specified 300
Created a chunk of size 312, which is longer than the specified 300
Created a chunk of size 421, which is longer than the specified 300
Created a chunk of size 494, which is longer than the specified 300
Created a chunk of size 350, which is longer than the specified 300
Created a chunk of size 744, which is longer than the specified 300
Created a chunk of size 555, which is longer than the specified 300
Created a chunk of size 453, which is longer than the specified 300
Created a chunk of size 540, which is longer tha

분할 전 문서 개수: 191
분할 후 문서 개수: 826


In [5]:
# SentenceTransformer 모델 로드
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# SentenceTransformer 적재를 위해 Document 객체에서 텍스트 추출
texts = [doc.page_content for doc in splitted_documents]

# 문서 임베딩 생성
embeddings = embedding_model.embed_documents(texts)

# FAISS 인덱스 생성
d = len(embeddings[0])  # 임베딩 차원 (예: 384차원)
index = faiss.IndexFlatL2(d)  # L2 거리 기반의 FAISS 인덱스 생성
index.add(np.array(embeddings))

# LangChain의 FAISS와 연결
docstore = InMemoryDocstore({i: Document(page_content=texts[i]) for i in range(len(texts))})
faiss_index = FAISS(embedding_function=embedding_model, index=index, docstore=docstore, index_to_docstore_id={i: i for i in range(len(texts))})

# FAISS 인덱스가 생성되었습니다.
print("Documents added to the FAISS index.")

/var/folders/hw/9m3g7fvn4_l3rp2y473km9sm0000gn/T/ipykernel_1740/1829115815.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Documents added to the FAISS index.


In [16]:
embeddings = embedding_model

database = faiss_index

query = "2023년 통화정책의 가장 중요한 목표와 고려사항은?"

results = faiss_index.similarity_search(query, k=5) 

context = "Relevant documents:\n"  #조회를 통해 얻은 정보를 저장할 변수 초기화

for i, result in enumerate(results):
    context += f"""
-----------------------------------
Doc {i+1}:
{result.page_content}
"""

prompt = PromptTemplate(
    template="""Answer the question based on the given information.

Information:
{document}

Question:{query} 
""",
    input_variables=['documents', 'query']
)

prompt = prompt.format(document=context, query=query)
print(prompt)

Answer the question based on the given information.

Information:
Relevant documents:

-----------------------------------
Doc 1:
한국은행 통합별관 준공 및 재입주 기념식
(2023년 4월 24일)
자료: 한국은행

-----------------------------------
Doc 2:
화폐사랑 콘텐츠 공모전 시상식 
              (2023년 11월 21일)
자료: 한국은행

-----------------------------------
Doc 3:
2023년말 현재 한국은행, 
금융기관, 유관기관 등 총 32개 기관이 참여하고 있으며, 산하에 금융정보화 공동추진사업의 선정, 금융표준화 등의 사전 심
의를 위한 실무협의회를 두고 있다.

-----------------------------------
Doc 4:
기업의전반적인채무상환능력은
전기전자, 석유화학, 건설등업종의업황부진과
금리상승등의영향으로저하되면서기업대출
연체율이상승72)하였다.

-----------------------------------
Doc 5:
먼저 일부 위원은 금융안정 책무수행에 
있어 당행의 정책권한과 책임을 정립해 
나갈 필요가 있으며, 이를 바탕으로 당


Question:2023년 통화정책의 가장 중요한 목표와 고려사항은? 



In [10]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers

model_id = "beomi/gemma-ko-2b"
dtype = torch.bfloat16

tokenizer = AutoTokenizer.from_pretrained("beomi/gemma-ko-2b")
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="mps",
    torch_dtype=dtype,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [17]:
inputs = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
outputs = model.generate(input_ids=inputs.to(model.device), max_new_tokens=150)
print(tokenizer.decode(outputs[0]))

Answer the question based on the given information.

Information:
Relevant documents:

-----------------------------------
Doc 1:
한국은행 통합별관 준공 및 재입주 기념식
(2023년 4월 24일)
자료: 한국은행

-----------------------------------
Doc 2:
화폐사랑 콘텐츠 공모전 시상식 
              (2023년 11월 21일)
자료: 한국은행

-----------------------------------
Doc 3:
2023년말 현재 한국은행, 
금융기관, 유관기관 등 총 32개 기관이 참여하고 있으며, 산하에 금융정보화 공동추진사업의 선정, 금융표준화 등의 사전 심
의를 위한 실무협의회를 두고 있다.

-----------------------------------
Doc 4:
기업의전반적인채무상환능력은
전기전자, 석유화학, 건설등업종의업황부진과
금리상승등의영향으로저하되면서기업대출
연체율이상승72)하였다.

-----------------------------------
Doc 5:
먼저 일부 위원은 금융안정 책무수행에 
있어 당행의 정책권한과 책임을 정립해 
나갈 필요가 있으며, 이를 바탕으로 당


Question:2023년 통화정책의 가장 중요한 목표와 고려사항은? 
(1) 통화정책의 목표는 
(2) 통화정책의 고려사항은 
-----------------------------------
Doc 1:
한국은행 통합별관 준공 및 재입주 기념식
(2023년 4월 24일)
자료: 한국은행

-----------------------------------
Doc 2:
화폐사랑 콘텐츠 공모전 시상식 
              (2023년 11월 21일)
자료: 한국은행

-----------------------------------
Doc 3:
2023년말 현재 한국은행, 
금융기관, 유
